In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from datetime import timedelta

In [2]:
filename1 ='../../data/stock_price/TSLA_30m_2y.csv'
filename2 ='../../data/stock_price/TSLA.csv'

tsla_hour = pd.read_csv(filename1)
tsla_day = pd.read_csv(filename2)

tsla_hour.shape, tsla_day.shape

((5089, 8), (720, 7))

In [3]:
tsla_hour.head()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2016-05-23 09:30:00-04:00,220.06,222.30,217.16,222.14,1770979,0,0
1,2016-05-23 10:30:00-04:00,221.92,222.60,219.26,219.49,678668,0,0
2,2016-05-23 11:30:00-04:00,219.51,219.53,217.66,218.47,601477,0,0
3,2016-05-23 12:30:00-04:00,218.57,219.04,217.41,217.49,439506,0,0
4,2016-05-23 13:30:00-04:00,217.44,217.60,216.19,217.49,514489,0,0


In [4]:
tsla_hour.drop(['Dividends', 'Stock Splits'], axis = 1, inplace = True)

In [5]:
tsla_hour.tail()

,Datetime,Open,High,Low,Close,Volume
5084,2019-04-16 11:30:00-04:00,271.16,273.70,271.06,272.47,999939
5085,2019-04-16 12:30:00-04:00,272.55,274.74,271.81,273.90,866762
5086,2019-04-16 13:30:00-04:00,273.91,274.97,273.21,274.79,750558
5087,2019-04-16 14:30:00-04:00,274.75,275.00,271.81,272.26,675595
5088,2019-04-16 15:30:00-04:00,272.27,273.67,271.66,273.31,707819


In [6]:
tsla_day.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
715,2019-03-27,268.750000,275.369995,268.179993,274.829987,274.829987,8779200
716,2019-03-28,277.160004,280.329987,275.100006,278.619995,278.619995,6774100
717,2019-03-29,278.700012,280.160004,274.500000,279.859985,279.859985,5991300
718,2019-04-01,282.619995,289.200012,281.279999,289.179993,289.179993,8110400
719,2019-04-02,288.299988,289.440002,283.880005,285.880005,285.880005,5478900


In [7]:
tsla_hour['Datetime'] = pd.to_datetime(tsla_hour["Datetime"], utc = True)
tsla_hour['Date'] = pd.to_datetime(tsla_hour['Datetime']).dt.date
tsla_hour['Date'] = pd.to_datetime(tsla_hour['Date'])
tsla_hour.rename(columns = {'Volume':'Hour_volume'}, inplace = True)
tsla_day['Date'] = pd.to_datetime(tsla_day["Date"])

In [8]:
tsla_hour_new = tsla_hour[['Date', 'Datetime','Hour_volume']]
tsla_day_new = tsla_day[['Date','Volume']]

In [9]:
tsla_hour_new.head()

,Date,Datetime,Hour_volume
0,2016-05-23,2016-05-23 13:30:00+00:00,1770979
1,2016-05-23,2016-05-23 14:30:00+00:00,678668
2,2016-05-23,2016-05-23 15:30:00+00:00,601477
3,2016-05-23,2016-05-23 16:30:00+00:00,439506
4,2016-05-23,2016-05-23 17:30:00+00:00,514489


In [10]:
tsla_day_new.head()

,Date,Volume
0,2016-05-23,5093200
1,2016-05-24,3013800
2,2016-05-25,3126800
3,2016-05-26,4064000
4,2016-05-27,3650300


In [11]:
merge = tsla_day_new.join(tsla_hour_new.set_index('Date'), on='Date')
merge.tail(10)

,Date,Volume,Datetime,Hour_volume
718,2019-04-01,8110400,2019-04-01 17:30:00+00:00,994985
718,2019-04-01,8110400,2019-04-01 18:30:00+00:00,994315
718,2019-04-01,8110400,2019-04-01 19:30:00+00:00,1093165
719,2019-04-02,5478900,2019-04-02 13:30:00+00:00,1497072
719,2019-04-02,5478900,2019-04-02 14:30:00+00:00,721978
719,2019-04-02,5478900,2019-04-02 15:30:00+00:00,699785
719,2019-04-02,5478900,2019-04-02 16:30:00+00:00,497682
719,2019-04-02,5478900,2019-04-02 17:30:00+00:00,355504
719,2019-04-02,5478900,2019-04-02 18:30:00+00:00,641972
719,2019-04-02,5478900,2019-04-02 19:30:00+00:00,855794


In [12]:
merge.dropna(inplace = True)
merge.shape

(5019, 4)

In [13]:
merge.reset_index(drop=True, inplace=True)

In [14]:
merge.tail()

,Date,Volume,Datetime,Hour_volume
5014,2019-04-02,5478900,2019-04-02 15:30:00+00:00,699785
5015,2019-04-02,5478900,2019-04-02 16:30:00+00:00,497682
5016,2019-04-02,5478900,2019-04-02 17:30:00+00:00,355504
5017,2019-04-02,5478900,2019-04-02 18:30:00+00:00,641972
5018,2019-04-02,5478900,2019-04-02 19:30:00+00:00,855794


In [15]:
# include trading hour
merge['Trading_Hour'] = merge["Datetime"].dt.hour-12

In [16]:
# shift daylight date
daylight_dates = merge[merge.Trading_Hour==8].Datetime
daylight_dates = list(daylight_dates.dt.date)
merge[merge['Date'].isin(daylight_dates)]
new_trading_hours = merge[merge['Date'].isin(daylight_dates)]['Trading_Hour'] - 1
merge.loc[merge['Date'].isin(daylight_dates), 'Trading_Hour'] = new_trading_hours

In [17]:
merge.groupby('Trading_Hour')['Trading_Hour'].count()

Trading_Hour
1    716
2    720
3    719
4    717
5    717
6    716
7    714
Name: Trading_Hour, dtype: int64

# find the volume = 0 dates

In [18]:
merge_0 = merge[merge['Hour_volume'] == 0]

In [19]:
merge_0.groupby('Date')['Hour_volume'].count()[merge_0.groupby('Date')['Hour_volume'].count() != 1]

Date
2017-02-23    2
2017-03-16    2
2017-03-22    2
2017-03-24    2
2017-03-29    2
2017-04-03    2
2017-04-05    3
2017-04-11    2
2017-04-24    2
2017-05-01    2
2017-05-09    4
2017-05-15    2
2017-05-19    2
2017-06-01    2
2017-06-12    2
2017-06-16    2
2017-06-20    2
Name: Hour_volume, dtype: int64

In [20]:
import datetime

In [21]:
merge[merge['Date'] == datetime.date(2017,5,9)]

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


,Date,Volume,Datetime,Hour_volume,Trading_Hour
1691,2017-05-09,9676500,2017-05-09 13:30:00+00:00,2799955,1
1692,2017-05-09,9676500,2017-05-09 14:30:00+00:00,0,2
1693,2017-05-09,9676500,2017-05-09 15:30:00+00:00,0,3
1694,2017-05-09,9676500,2017-05-09 16:30:00+00:00,881621,4
1695,2017-05-09,9676500,2017-05-09 17:30:00+00:00,0,5
1696,2017-05-09,9676500,2017-05-09 18:30:00+00:00,0,6
1697,2017-05-09,9676500,2017-05-09 19:30:00+00:00,947168,7


# impute data for normal dates (with 7 data points)

In [22]:
def impute(hourdat):
    
    dtotal = hourdat.Volume.iloc[0]
    htotal = sum(hourdat.Hour_volume)
    # replace extreme high value with 0 and impute with average later
    hourdat.loc[hourdat.Hour_volume > dtotal,'Hour_volume'] = 0
    
    if len(hourdat) == 7:
        diff = dtotal - htotal
        icell = sum(hourdat.Hour_volume == 0)
        # impute by diff of total devide by all empty cell
        hourdat['Hour_volume'] = hourdat.Hour_volume.replace(0, diff/icell)  
               
    return hourdat

In [23]:
merge = merge.groupby('Date').apply(lambda x: impute(x))

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in long_scalars
  if sys.path[0] == '':
/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in long_scalars
  if sys.path[0] == '':
/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in long_scalars
  if sys.path[0] == '':
/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in long_scalars
  if sys.path[0] == '':
/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in long_scalars
  if sys.path[0] == '':
/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in long_scalars
  if sys.path[0] == '':
/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarni

In [24]:
merge[merge['Date'] == datetime.date(2017,5,9)]

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


,Date,Volume,Datetime,Hour_volume,Trading_Hour
1691,2017-05-09,9676500,2017-05-09 13:30:00+00:00,2799955.0,1
1692,2017-05-09,9676500,2017-05-09 14:30:00+00:00,1261939.0,2
1693,2017-05-09,9676500,2017-05-09 15:30:00+00:00,1261939.0,3
1694,2017-05-09,9676500,2017-05-09 16:30:00+00:00,881621.0,4
1695,2017-05-09,9676500,2017-05-09 17:30:00+00:00,1261939.0,5
1696,2017-05-09,9676500,2017-05-09 18:30:00+00:00,1261939.0,6
1697,2017-05-09,9676500,2017-05-09 19:30:00+00:00,947168.0,7


# find the missing dates

In [25]:
missing_date = merge.groupby(['Date'])['Trading_Hour'].count()[merge.groupby(['Date'])['Trading_Hour'].count() != 7].reset_index()
missing_date


,Date,Trading_Hour
0,2016-11-25,4
1,2017-07-03,4
2,2017-11-24,4
3,2018-07-03,4
4,2018-08-07,6
5,2018-11-23,4
6,2018-11-29,5
7,2018-12-24,4


In [26]:
missing_date["Date"] = pd.to_datetime(missing_date["Date"])

In [27]:
for i in missing_date['Date']:
    temp = merge[merge['Date'] == i]
    print(i, temp[temp['Hour_volume'] == 0].shape[0])

2016-11-25 00:00:00 1
2017-07-03 00:00:00 0
2017-11-24 00:00:00 1
2018-07-03 00:00:00 1
2018-08-07 00:00:00 0
2018-11-23 00:00:00 1
2018-11-29 00:00:00 1
2018-12-24 00:00:00 0


In [29]:
for i in missing_date['Date']:
    end = np.where(merge['Date'] == i)[0][0]
    start = end - 5*7
    past = merge.iloc[start:end]
    current = merge[merge['Date'] == i]
    volume = current['Volume'].iloc[0]
    hour = current['Datetime'].dt.hour.iloc[0] - current['Trading_Hour'].iloc[0]
    trading_hour = current['Trading_Hour'].tolist()
           
    # add rows
    for j in range(1,8):        
        if j not in trading_hour:
            print(i, j)
            new  = past[past['Trading_Hour'] == j]['Hour_volume']
            hour_vol = sum(new)/len(new)
            datetime = i + timedelta(hours = int(j + hour)) + timedelta(minutes = 30)
            merge = merge.append(pd.Series([i, volume, datetime, hour_vol, j], index=merge.columns), ignore_index=True)
    

2016-11-25 00:00:00 1
2016-11-25 00:00:00 6
2016-11-25 00:00:00 7
2017-07-03 00:00:00 4
2017-07-03 00:00:00 6
2017-07-03 00:00:00 7
2017-11-24 00:00:00 1
2017-11-24 00:00:00 5
2017-11-24 00:00:00 7
2018-07-03 00:00:00 4
2018-07-03 00:00:00 6
2018-07-03 00:00:00 7
2018-08-07 00:00:00 6
2018-11-23 00:00:00 1
2018-11-23 00:00:00 5
2018-11-23 00:00:00 7
2018-11-29 00:00:00 3
2018-11-29 00:00:00 4
2018-12-24 00:00:00 1
2018-12-24 00:00:00 5
2018-12-24 00:00:00 7


In [31]:
import datetime

In [32]:
merge[merge['Date'] == datetime.date(2017,11,24)]

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


,Date,Volume,Datetime,Hour_volume,Trading_Hour
2661,2017-11-24,3244100,2017-11-24 14:30:00+00:00,1148395.0,2
2662,2017-11-24,3244100,2017-11-24 15:30:00+00:00,698494.0,3
2663,2017-11-24,3244100,2017-11-24 16:30:00+00:00,512739.0,4
2664,2017-11-24,3244100,2017-11-24 18:00:00+00:00,0.0,6
5025,2017-11-24,3244100,2017-11-24 13:30:00,2585457.2,1
5026,2017-11-24,3244100,2017-11-24 17:30:00,615202.2,5
5027,2017-11-24,3244100,2017-11-24 19:30:00,763468.2,7


In [34]:
merge['Datetime'] = pd.to_datetime(merge["Datetime"], utc = True)
merge.tail(20)

,Date,Volume,Datetime,Hour_volume,Trading_Hour
5020,2016-11-25,2366100,2016-11-25 18:30:00+00:00,495235.6,6
5021,2016-11-25,2366100,2016-11-25 19:30:00+00:00,609738.2,7
5022,2017-07-03,6305400,2017-07-03 16:30:00+00:00,700050.4,4
5023,2017-07-03,6305400,2017-07-03 18:30:00+00:00,596364.8,6
5024,2017-07-03,6305400,2017-07-03 19:30:00+00:00,797168.4,7
5025,2017-11-24,3244100,2017-11-24 13:30:00+00:00,2585457.2,1
5026,2017-11-24,3244100,2017-11-24 17:30:00+00:00,615202.2,5
5027,2017-11-24,3244100,2017-11-24 19:30:00+00:00,763468.2,7
5028,2018-07-03,12282600,2018-07-03 16:30:00+00:00,1086472.0,4
5029,2018-07-03,12282600,2018-07-03 18:30:00+00:00,1018114.2,6


In [33]:
merge.shape

(5040, 5)

In [ ]:
import datetime

In [35]:
merge[merge['Date'] == datetime.date(2016,11,25)]

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


,Date,Volume,Datetime,Hour_volume,Trading_Hour
910,2016-11-25,2366100,2016-11-25 14:30:00+00:00,0.0,2
911,2016-11-25,2366100,2016-11-25 15:30:00+00:00,595738.0,3
912,2016-11-25,2366100,2016-11-25 16:30:00+00:00,445903.0,4
913,2016-11-25,2366100,2016-11-25 17:30:00+00:00,537028.0,5
5019,2016-11-25,2366100,2016-11-25 13:30:00+00:00,1406823.6,1
5020,2016-11-25,2366100,2016-11-25 18:30:00+00:00,495235.6,6
5021,2016-11-25,2366100,2016-11-25 19:30:00+00:00,609738.2,7


In [36]:
a = merge[merge['Hour_volume'] == 0]
a

,Date,Volume,Datetime,Hour_volume,Trading_Hour
910,2016-11-25,2366100,2016-11-25 14:30:00+00:00,0.0,2
2664,2017-11-24,3244100,2017-11-24 18:00:00+00:00,0.0,6
3718,2018-07-03,12282600,2018-07-03 17:00:00+00:00,0.0,5
4411,2018-11-23,4202600,2018-11-23 14:30:00+00:00,0.0,2
4436,2018-11-29,3080700,2018-11-29 14:30:00+00:00,0.0,1


In [46]:
a.iloc[0].name

910

In [50]:
for i in range(a.shape[0]):
    end = a.iloc[i].name
    start = end - 5*7
    past = merge.iloc[start:end]
    current = merge[merge['Date'] == a.iloc[i]['Date']]
    h = a.iloc[i]['Trading_Hour']
    new = past[past['Trading_Hour'] == h]['Hour_volume']
    hour_vol = sum(new)/len(new)
    #print(hour_vol)
    merge.loc[end, 'Hour_volume'] = hour_vol
    #print(merge.iloc[end]['Hour_volume'])

In [51]:
merge[merge['Date'] == datetime.date(2017,11,24)]

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


,Date,Volume,Datetime,Hour_volume,Trading_Hour
2661,2017-11-24,3244100,2017-11-24 14:30:00+00:00,674242.6,2
2662,2017-11-24,3244100,2017-11-24 15:30:00+00:00,698494.0,3
2663,2017-11-24,3244100,2017-11-24 16:30:00+00:00,512739.0,4
2664,2017-11-24,3244100,2017-11-24 18:00:00+00:00,674242.6,6
5025,2017-11-24,3244100,2017-11-24 13:30:00+00:00,2585457.2,1
5026,2017-11-24,3244100,2017-11-24 17:30:00+00:00,615202.2,5
5027,2017-11-24,3244100,2017-11-24 19:30:00+00:00,763468.2,7


In [52]:
merge.sort_values(by = ['Date', 'Trading_Hour'], inplace = True)
merge.reset_index(inplace = True, drop = True)

In [53]:
merge.head(20)

,Date,Volume,Datetime,Hour_volume,Trading_Hour
0,2016-05-23,5093200,2016-05-23 13:30:00+00:00,1770979.0,1
1,2016-05-23,5093200,2016-05-23 14:30:00+00:00,678668.0,2
2,2016-05-23,5093200,2016-05-23 15:30:00+00:00,601477.0,3
3,2016-05-23,5093200,2016-05-23 16:30:00+00:00,439506.0,4
4,2016-05-23,5093200,2016-05-23 17:30:00+00:00,514489.0,5
5,2016-05-23,5093200,2016-05-23 18:30:00+00:00,332856.0,6
6,2016-05-23,5093200,2016-05-23 19:30:00+00:00,500023.0,7
7,2016-05-24,3013800,2016-05-24 13:30:00+00:00,1088736.0,1
8,2016-05-24,3013800,2016-05-24 14:30:00+00:00,565361.0,2
9,2016-05-24,3013800,2016-05-24 15:30:00+00:00,282089.0,3


In [54]:
merge.groupby('Trading_Hour')['Trading_Hour'].count()

Trading_Hour
1    720
2    720
3    720
4    720
5    720
6    720
7    720
Name: Trading_Hour, dtype: int64

In [55]:
merge[merge['Hour_volume'] == 0]

,Date,Volume,Datetime,Hour_volume,Trading_Hour


In [ ]:
#sum_volume = merge.groupby(['Date'])['Hour_volume'].sum().rename('Sum_volume').reset_index()

In [ ]:
#merge_new = merge.join(sum_volume.set_index('Date'), on='Date')

# Calculate volume percentage

In [56]:
roll_volume = merge['Hour_volume'].rolling(window=7).sum().rename('Rolling_volume')
merge['Roll_volume'] = roll_volume

In [57]:
moving_average_5 = merge['Roll_volume'].rolling(window = 5).mean().rename('Moving_average_5')
merge['Moving_average_5'] = moving_average_5

In [58]:
merge['Volume_perc'] = merge['Hour_volume']/merge['Moving_average_5']

In [59]:
merge.rename(columns = {'Volume':'Daily_volume'}, inplace = True)

In [60]:
merge.head(20)

,Date,Daily_volume,Datetime,Hour_volume,Trading_Hour,Roll_volume,Moving_average_5,Volume_perc
0,2016-05-23,5093200,2016-05-23 13:30:00+00:00,1770979.0,1,NaN,NaN,NaN
1,2016-05-23,5093200,2016-05-23 14:30:00+00:00,678668.0,2,NaN,NaN,NaN
2,2016-05-23,5093200,2016-05-23 15:30:00+00:00,601477.0,3,NaN,NaN,NaN
3,2016-05-23,5093200,2016-05-23 16:30:00+00:00,439506.0,4,NaN,NaN,NaN
4,2016-05-23,5093200,2016-05-23 17:30:00+00:00,514489.0,5,NaN,NaN,NaN
5,2016-05-23,5093200,2016-05-23 18:30:00+00:00,332856.0,6,NaN,NaN,NaN
6,2016-05-23,5093200,2016-05-23 19:30:00+00:00,500023.0,7,4837998.0,NaN,NaN
7,2016-05-24,3013800,2016-05-24 13:30:00+00:00,1088736.0,1,4155755.0,NaN,NaN
8,2016-05-24,3013800,2016-05-24 14:30:00+00:00,565361.0,2,4042448.0,NaN,NaN
9,2016-05-24,3013800,2016-05-24 15:30:00+00:00,282089.0,3,3723060.0,NaN,NaN


In [61]:
merge = merge.iloc[14:]

In [62]:
merge.reset_index(inplace = True, drop = True)

In [63]:
merge.head(20)

,Date,Daily_volume,Datetime,Hour_volume,Trading_Hour,Roll_volume,Moving_average_5,Volume_perc
0,2016-05-25,3126800,2016-05-25 13:30:00+00:00,772125.0,1,2623287.0,3082185.6,0.250512
1,2016-05-25,3126800,2016-05-25 14:30:00+00:00,706932.0,2,2764858.0,2925038.0,0.241683
2,2016-05-25,3126800,2016-05-25 15:30:00+00:00,271832.0,3,2754601.0,2839565.6,0.095730
3,2016-05-25,3126800,2016-05-25 16:30:00+00:00,288918.0,4,2776477.0,2771824.2,0.104234
4,2016-05-25,3126800,2016-05-25 17:30:00+00:00,440561.0,5,3071182.0,2798081.0,0.157451
5,2016-05-25,3126800,2016-05-25 18:30:00+00:00,264957.0,6,3070062.0,2887436.0,0.091762
6,2016-05-25,3126800,2016-05-25 19:30:00+00:00,329208.0,7,3074533.0,2949371.0,0.111620
7,2016-05-26,4064000,2016-05-26 13:30:00+00:00,1020114.0,1,3322522.0,3062955.2,0.333049
8,2016-05-26,4064000,2016-05-26 14:30:00+00:00,887124.0,2,3502714.0,3208202.6,0.276517
9,2016-05-26,4064000,2016-05-26 15:30:00+00:00,506720.0,3,3737602.0,3341486.6,0.151645


In [64]:
merge.groupby('Trading_Hour')['Trading_Hour'].count()

Trading_Hour
1    718
2    718
3    718
4    718
5    718
6    718
7    718
Name: Trading_Hour, dtype: int64

In [65]:
import pickle

In [66]:
with open('../../data/features/2019_05_16_TSLA_y_06_yishuang.pickle', 'wb') as handle:
        pickle.dump(merge, handle, protocol=pickle.HIGHEST_PROTOCOL)